In [1]:
def workflow_batch_process_folder_measure_intensity(image_folder_location, labels_folder_location):
    """
    This functions goes through all .tif image files in a specified image folder 
    and corresponding label images in another labels folder. 
    It loads the images and corresponding labels, and measures min, mean and max intensity of all labels.
    The function returns a dataframe with five columns: min_intensity, mean_intensity, max_intensity, label and filename.
    """
    import os
    import pandas as pd
    from skimage.io import imread
    from skimage.measure import regionprops
    import numpy as np

    supported_fileendings = [".tif", ".jpg", ".png"]    
    file_list = [fn for fn in os.listdir(image_folder_location) if str(fn[-4:]) in supported_fileendings]

    result = []

    for filename in file_list:
        image = imread(image_folder_location + filename)[...,0]
        labels = imread(labels_folder_location + filename)

        stats = regionprops(labels, intensity_image=image)

        for s in stats:
            result.append({
                "filename":filename,
                "label":s.label,
                "min_intensity":s.min_intensity,
                "mean_intensity":s.mean_intensity,
                "max_intensity":s.max_intensity,
            })

    return pd.DataFrame(result)

In [2]:
def check(candidate):
    label_stats = candidate("../example_data/S-BIAD634/images/", "../example_data/S-BIAD634/groundtruth/")

    assert label_stats['label'].max() == 398
    assert label_stats['min_intensity'].min() == 7
    assert label_stats['max_intensity'].max() == 255

    assert label_stats.size == 9035
    
    assert abs(label_stats['mean_intensity'].max() - 186) < 1

    assert len(label_stats.columns) == 5
    assert len(label_stats['mean_intensity']) == 1807

    assert "filename" in label_stats.columns
    assert "label" in label_stats.columns
    assert "min_intensity" in label_stats.columns
    assert "mean_intensity" in label_stats.columns
    assert "max_intensity" in label_stats.columns

In [3]:
check(workflow_batch_process_folder_measure_intensity)